In [16]:
import pandas as pd

In [17]:
contract_address = pd.read_csv('./data/contract_address_list.csv')

In [18]:
contract_address_set = set(contract_address['contract_address'])

In [94]:
len(contract_address_set)

16562632

In [20]:
contract_info = pd.read_csv('ContractInfo_buDecreate.csv')

D:\ProgramData\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
len(set(contract_info['address']))

5706058

In [22]:
len(contract_info)

33229960

In [23]:
new_contract_address_set = set(contract_info['address'])

In [6]:
import pickle
with open('./decreate_contract_address_set.dat', 'wb') as f:
    pickle.dump(decreate_contract_address_set,f)

## 数据主体

### 分片读取 + 选取时间切片 +  剔除合约记录

In [57]:
chunker = pd.read_csv('./data/InternalEtherTransaction_queTimestamp.csv'
                                  ,usecols=['blockNumber', 'from', 'to', 'value']
                                  ,dtype={'blockNumber': int}
                                  ,chunksize=1000000)


In [ ]:
for item in chunker:
    print(item.iloc[0]['blockNumber'])

In [58]:
filtered_part = pd.DataFrame()
for item in chunker:
    print(str(item.iloc[0]['blockNumber'])+' ' +str(len(filtered_part)), end='\r')
    if (item.iloc[-1]['blockNumber'] <= 2000000): continue
    if (item.iloc[0]['blockNumber'] > 7000000): break
    tmp = item[~(item['from'].isin(new_contract_address_set) | item['to'].isin(new_contract_address_set))]
    filtered_part = pd.concat([filtered_part, tmp])

In [59]:
#len(internal_transaction)

In [61]:
#filtered_part[(filtered_part['from'] == '0x0022a370cdebaff99746b8a1311a8d9734bf3a28') | ((filtered_part['to'] == '0x0022a370cdebaff99746b8a1311a8d9734bf3a28'))]

In [11]:
#internal_transaction[(internal_transaction['from'] == '0x0022a370cdebaff99746b8a1311a8d9734bf3a28') | ((internal_transaction['to'] == '0x0022a370cdebaff99746b8a1311a8d9734bf3a28'))]

,blockNumber,transactionHash,from,to,value
176257010,5547183,0x6fa40cf0bc1124719c371e474a9743ac7eaa2b781afd...,0xf78c208e3578be35bc1f5468cd2559d70a0b72f0,0x0022a370cdebaff99746b8a1311a8d9734bf3a28,20000000000000000
186780881,5668104,0x5cb265fe0bdcc0d29da91013e0b2df28cf4f36a8f26a...,0xf78c208e3578be35bc1f5468cd2559d70a0b72f0,0x0022a370cdebaff99746b8a1311a8d9734bf3a28,50000000000000000
195909343,5781106,0x127dd7c5419b1fc8993cd90778587cc092c80054953c...,0xf78c208e3578be35bc1f5468cd2559d70a0b72f0,0x0022a370cdebaff99746b8a1311a8d9734bf3a28,3000000000000000000
213519571,6044520,0x8f3d52db088ef145966986ee3b43d7abef13efb7f043...,0x0022a370cdebaff99746b8a1311a8d9734bf3a28,0x1158dcb8a5a37e8fd0f7bb6dd8e74538420079ab,1265720000000000000


In [57]:
#without_contract_data = internal_transaction[~(internal_transaction['from'].isin(contract_address_set) | internal_transaction['to'].isin(contract_address_set))]

In [62]:
from_counts = filtered_part['from'].value_counts()
to_counts = filtered_part['to'].value_counts()

In [8]:
#part_data = internal_transaction.sample(frac=0.001)

In [61]:
#from_counts = without_contract_data['from'].value_counts()
#to_counts = without_contract_data['to'].value_counts()

In [69]:
phishing_label = pd.read_csv('./data/phishing_label.csv')
phishing_address_set = set(phishing_label['address'])
phising_lowercase_address_set = set(map(lambda x: x.lower(), phishing_address_set))

In [70]:
#len(phising_lowercase_address_set.intersection(set(tmp_df['index'])))

2162

In [71]:
phishing_data = filtered_part[(filtered_part['from'].isin(phising_lowercase_address_set) | filtered_part['to'].isin(phising_lowercase_address_set))]

In [ ]:
#phishing_data.sort_values(by='blockNumber')

In [ ]:
#phishing_data['blockNumber'].apply(lambda x: str(x)[:2]).value_counts()

In [65]:
tmp_df = pd.DataFrame(to_counts).reset_index()

In [67]:
tmp_df[tmp_df['to'] < 1000]['to'].sum()

164083636

In [85]:
import time

## 选择数据

In [117]:
#200w开始才观测到黑样本，700w为18年末
#filtered_data = without_contract_data[(without_contract_data['blockNumber'] > 2000000) & (without_contract_data['blockNumber'] < 7000000)]

In [124]:
#from_counts = internal_transaction['from'].value_counts()
#to_counts = internal_transaction['to'].value_counts()

In [72]:
tmp_df = pd.DataFrame(from_counts).reset_index()
tmp_df = tmp_df[(tmp_df['from'] < 1000)]
from_low_freq_set = set(tmp_df['index'])

tmp_df = pd.DataFrame(to_counts).reset_index()
tmp_df = tmp_df[(tmp_df['to'] < 1000)]
to_low_freq_set = set(tmp_df['index'])
#from to 均 < 1000 较为宽松的条件
low_freq_set = from_low_freq_set.intersection(to_low_freq_set)

In [73]:
len(low_freq_set)

33841810

In [81]:
len(phising_lowercase_address_set)

2880

In [74]:
filtered_low_freq_data =  filtered_part[(filtered_part['from'].isin(low_freq_set)) & (filtered_part['to'].isin(low_freq_set))]

In [88]:
filtered_low_freq_data.to_csv('./data/filtered_data.csv',index = False)

In [76]:
filtered_from_low_freq_data_address = set(filtered_low_freq_data['from'])
filtered_to_low_freq_data_address = set(filtered_low_freq_data['to'])

In [77]:
filtered_low_freq_data_address = filtered_from_low_freq_data_address.union(filtered_to_low_freq_data_address)

In [86]:
len(phising_lowercase_address_set.intersection(filtered_low_freq_data_address))

1937

In [137]:
#len(phishing_set.intersection(filtered_low_freq_data_address))

1303

In [87]:
phishing_data.to_csv('./data/all_phishing_data.csv', index=False)

In [93]:
len(set(filtered_low_freq_data['from']).union(set(filtered_low_freq_data['to'])))

27680168